In [19]:
import Indices
from Entry.baseentry import BaseEntry
import numpy as np
import time
import sys

In [20]:
DIM = 20
NUM_VECTORS = 20
def random_vector() -> np.array:
    return np.float32(np.random.rand(DIM))

def time_function(func, *args, **kwargs) -> float:
    start = time.time()
    func(*args, **kwargs)
    end = time.time()
    return end - start

def get_mem_usage(obj) -> str:
    size_bytes = sys.getsizeof(obj)
    size_mb = size_bytes / (1024 * 1024)
    return f"{size_mb} MB"

def check_equality(a: np.array, b: np.array) -> None:
    np.testing.assert_array_equal(a,b)

q = random_vector()
emb = [random_vector() for i in range(NUM_VECTORS)]

Usage

In [31]:
root = Indices.SVMRootIndex("root", 10)

In [32]:
root.create_child_level(["a", "b"], Indices.SVMInnerIndex, keys=[random_vector() for i in range(2)],max_children = NUM_VECTORS)
root.children["a"].create_child_level(["a1", "a2"], Indices.HNSWLeafIndex, keys=[random_vector() for i in range(2)],max_children = NUM_VECTORS, dim = DIM)
root.children["b"].create_child_level(["b1", "b2"], Indices.HNSWLeafIndex, keys=[random_vector() for i in range(2)],max_children = NUM_VECTORS, dim = DIM)

In [33]:
embeddings = np.array([random_vector() for _ in range(NUM_VECTORS)])
embeddings = BaseEntry.from_arr(embeddings)

In [34]:
root.insert_all(embeddings, "a-a1")
root.insert_all(embeddings, "a-a2")
root.insert_all(embeddings, "b-b1")
root.insert_all(embeddings, "b-b2")

In [37]:
root.delete_child("c")

2


In [38]:
root.create_child(child_name="c",t = Indices.HNSWLeafIndex, keys=[q], max_children = NUM_VECTORS, dim = DIM, key = q)
root.insert_all(BaseEntry.from_arr([q for i in range(NUM_VECTORS)], metadata=[{"test": "test"} for i in range(NUM_VECTORS)]), "c")

In [ ]:
# root.create_child(child_name="e",t = Indices.SVMInnerIndex, random_vector(), max_children = NUM_VECTORS, dim = DIM, key = q)

In [39]:
print(root.get_schema())

SVMRootIndex(name = root, num_children = 3)
    HNSWLeafIndex(name = c, dim = 20, current_count = 20, searchable = True)
    SVMInnerIndex(name=b, num_children=2, searchable = True)
        HNSWLeafIndex(name = b2, dim = 20, current_count = 20, searchable = True)
        HNSWLeafIndex(name = b1, dim = 20, current_count = 20, searchable = True)
    SVMInnerIndex(name=a, num_children=2, searchable = True)
        HNSWLeafIndex(name = a2, dim = 20, current_count = 20, searchable = True)
        HNSWLeafIndex(name = a1, dim = 20, current_count = 20, searchable = True)



In [40]:
path, items, dists = root.search(q, k = 5)

In [41]:
path

'c-'

In [ ]:
[check_equality(item.data, q) for item in items]

[None, None, None, None, None]

In [ ]:
BaseEntry.from_arr([q for i in range(NUM_VECTORS)], metadata=[{"test": "test"} for i in range(NUM_VECTORS)])[0].data.dtype

dtype('float32')

In [ ]:
time_function(root.search, q, k = 5)

0.0021822452545166016

In [ ]:
root.save_to_disk()

In [ ]:
def f():
    root = Indices.FlatRootIndex.load_from_disk("root")
    root.search(q, k = 5)

time_function(f)

0.009336233139038086

In [ ]:
get_mem_usage(root)

'4.57763671875e-05 MB'